In [48]:
__author__ = 'stan'

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utility as u
from sklearn import preprocessing as pre

datasets=u.get_datasets()
join=u.cat_join()
cat_cols=u.get_cat_cols()
samples=u.get_sample_names()


stats=u.get_stats_features(join)
priors=u.get_prior_features(join)
datasets['stats']=stats
datasets['priors']=priors

join=u.empty_join(datasets)

/home/stan/Documents/dev/telstra/utility.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  stats[column][mask]=0


In [49]:
#convert tables of categorical classes to binary row feature vectors for each unique id. Continuous factors are not affected
for key, dataset in datasets.items():
    if key not in ['stats','prior']:
        tmp=pd.get_dummies(dataset,dummy_na=True)
        if key=='log_feature':
            cols=tmp.columns.values.tolist()
            cols.remove('volume')
            cols.remove('id')
            tmp[cols]=tmp[cols].mul(tmp['volume'],axis=0)
        if key=='severity_type':
            cols=tmp.columns.values.tolist()
            cols.remove('id')
            myvol=datasets['log_feature'].loc[:,['id','volume']]
            myvol['volume']=pd.cut(myvol['volume'],bins=[0,1,2,4,7,12,30,45,60,80,100,150,175,250,1310], \
                                               labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14])
            tmp=pd.merge(tmp,myvol,on='id',how='left')
            tmp[cols]=tmp[cols].mul(tmp['volume'],axis=0)
        tmp=tmp.groupby('id').sum()
        tmp['id']=tmp.index
        dataset=tmp
        datasets[key]=dataset

datasets['log_feature']['volume'] = pd.cut(datasets['log_feature']['volume'], bins=[0, 1, 2, 7, 1310], labels=[1, 2, 3, 4]).astype(str)
#joins features from different datasets into one dataframe, features only for ids that appear in train/test are extracted
#(i.e. some features may not be related to any id)
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='left')

join=join.set_index(['sample'])


In [50]:
del join['volume_y']

In [52]:
join=pd.get_dummies(join) #fixes dummies for location

#TODO: TRY REMOVING ROWS WITH MISSING DATA
#fills in missing data
join=join.fillna(value=0)

#creates features and targets for final training
features=join.columns.values.tolist()
not_features=['fault_severity', 'id']
for x in features:
    if "_nan" in x:
        features.remove(x)
        not_features.append(x)

for x in not_features:
    if x in features:
        features.remove(x)

train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

In [53]:
join.head()

,fault_severity,id,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,...,location_location 990,location_location 991,location_location 992,location_location 993,location_location 994,location_location 995,location_location 996,location_location 997,location_location 998,location_location 999
sample,,,,,,,,,,,,,,,,,,,,,
train,1,14121,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,0,9320,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,1,14394,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,1,8218,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,0,14804,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
#XGBOOST
import xgboost as xgb

sz=(train_features.values).shape

train_X = train_features.values[:int(sz[0] * 0.7), :]
train_Y = train_target.values[:int(sz[0]*0.7)]

test_X = train_features.values[int(sz[0] * 0.7):, :]
test_Y = train_target.values[int(sz[0] * 0.7):]

xg_train = xgb.DMatrix(train_X, label=train_Y,feature_names=train_features.columns) #
full_xg_train = xgb.DMatrix(train_features, label=train_target.values,feature_names=train_features.columns) #

xg_test = xgb.DMatrix(test_X, label=test_Y,feature_names=test_features.columns) #
full_xg_test = xgb.DMatrix(test_features,feature_names=test_features.columns) #

#TRAIN
# setup parameters for xgboost
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 1
param['gamma']= 4
param['max_depth'] = 8
param['min_child_weight']=4
param['max_delta_step']=4
#param['subsample']=0.5

param['nthread'] = 3
#param['subsample']=1
param['num_class'] = 3
param['eval_metric']='mlogloss'



watchlist = [(xg_train,'train'), (xg_test, 'test')]
num_round = 6
bst = xgb.train(param, xg_train, num_round, evals=watchlist);

print(xgb.cv(param, xg_train, num_round, nfold=3, metrics=['mlogloss'], seed=0))

#xgb.plot_importance(booster=bst)
#plt.show()

bst=xgb.train(param, full_xg_train, num_round, evals=watchlist)
test_response=bst.predict(full_xg_test)
print(xgb.cv(param, xg_train, num_round, nfold=3, metrics=['mlogloss'], seed = 0))



#save predictions to required format
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=join.loc['test']['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
out.to_csv('predictions.csv',index=False)

[0]	train-mlogloss:0.684024	test-mlogloss:0.694966
[1]	train-mlogloss:0.603394	test-mlogloss:0.629708
[2]	train-mlogloss:0.571256	test-mlogloss:0.598462
[3]	train-mlogloss:0.548238	test-mlogloss:0.579284
[4]	train-mlogloss:0.525788	test-mlogloss:0.564786
[5]	train-mlogloss:0.519680	test-mlogloss:0.558839
[0]	train-mlogloss:0.683927	test-mlogloss:0.674925
[1]	train-mlogloss:0.606244	test-mlogloss:0.596105
[2]	train-mlogloss:0.568419	test-mlogloss:0.559647
[3]	train-mlogloss:0.541473	test-mlogloss:0.530851
[4]	train-mlogloss:0.529373	test-mlogloss:0.519997
[5]	train-mlogloss:0.516254	test-mlogloss:0.509570


   test-mlogloss-mean  test-mlogloss-std  train-mlogloss-mean  \
0            0.703918           0.011187             0.688089   
1            0.627545           0.012823             0.604414   
2            0.596130           0.015369             0.568158   
3            0.580573           0.015542             0.547287   
4            0.568690           0.013421             0.530885   
5            0.564966           0.016431             0.525301   

   train-mlogloss-std  
0            0.006272  
1            0.008100  
2            0.008994  
3            0.010120  
4            0.009046  
5            0.008549  
   test-mlogloss-mean  test-mlogloss-std  train-mlogloss-mean  \
0            0.703918           0.011187             0.688089   
1            0.627545           0.012823             0.604414   
2            0.596130           0.015369             0.568158   
3            0.580573           0.015542             0.547287   
4            0.568690           0.013421           